#  Additional Cleanup

I will be running 3 seperate notebooks to do the last bit of clean up, the reason being is that it took a long time to pull all songs, so I broke the pulls down and started on the next step as I had it available.

Now that I have all my lyrics I need to clean up obscure characters such as \n & \ from the strings. I will also remove the [verse 1] [chorus] parts because the artist's names are listed in there and will throw my model off.Finally, I will create a binarized column on whether the artist self-announces at the beginning of the track, I will need to extract the &feat artist name from title to add to the artist column.

In [192]:
import pandas as pd
import numpy as np
import regex as re

import lyricsgenius
from requests.exceptions import Timeout

pd.options.display.max_colwidth = 300


In [2]:
df = pd.read_csv('./data/lyrics_cp2.csv', index_col=0)

In [3]:
df.head()

,artists,title,track_id,popularity,explicit,release_year,lyrics
18116,Lady Gaga,The Edge Of Glory,57F80059mWaWFvWG3tBTbO,66,False,2011,"[Verse 1]\nThere ain't no reason you and me should be alone\nTonight, yeah, baby\nTonight, yeah,..."
18117,Lady Gaga,Million Reasons,1rpDLMm4FjGqMz6N1dZZo8,54,False,2016,[Verse 1]\nYou're giving me a million reasons to let you go\nYou're giving me a million reasons ...
18118,Lady Gaga,You And I,1ADciGUxOaUbBFd0rpYx7d,62,False,2011,"[Verse 1: Lady Gaga]\nIt’s been a long time since I came around\nBeen a long time, but I’m back ..."
18119,Lady Gaga,Heal Me,5Ctq1DOZUWZqsQZXro2jf7,61,False,2018,[Verse 1]\nTreat me like your patient\nJust don't keep me waiting\nOr I'll just be wasted\nIn a ...
18120,Lady Gaga,John Wayne,1vwTGFOq34pA2d0VUikL3W,47,False,2016,"[Intro: Spoken]\nIt's like, I just love a cowboy\nYou know\nI'm just like, I just, I know, it's ..."


Making sure there are no duplicates of artist and title again.

In [4]:
df.duplicated(['artists', 'title']).value_counts()

False    10248
dtype: int64

In making sure the lyrics were added correctly I noticed that for all songs that have (feat. artist) in the song title, the lyrics did not get pulled properly.
I will call lyricsgenius again to fix this.

In [5]:
df[df['title'].str.contains("feat.")]

,artists,title,track_id,popularity,explicit,release_year,lyrics
18136,Beyoncé,Video Phone - Extended Remix featuring Lady Gaga,7mzQFrKyyde7StPaXLss4K,34,False,2009,"[Intro: Beyoncé]\nUh, uh, uh, uh\nUh, uh, uh, uh\nUh, uh, uh, uh\nShawty, whatcho name is?\n\n[P..."
18238,Johnny Flynn,The Water - feat. Laura Marling,69FraOspp1B24ZFBqlUaoV,37,False,2010,"[Verse 1]\nAll that I have is a river\nThe river is always my home\nLord, take me away, for I ju..."
18260,Miranda Lambert,Smokin' and Drinkin' (feat. Little Big Town),13keyz9ikBe6ZpRasw7l4X,49,False,2014,[Verse 1]\nIt was one of those times what a real good time felt like a long time ago\nIt was one...
18270,Green Day,Ordinary World (feat. Miranda Lambert),3cASDwoemZJdsqLq11eD1n,40,False,2017,[Verse 1: Billie Joe Armstrong (with Miranda Lambert)]\nWhere can I find the city of shining lig...
18534,OneRepublic,Start Again (feat. Logic),1Nj2oBISdFDSF8eDuXcALz,65,False,2018,Pharrell Williams & Camila Cabello - Sangria Wine\nClean Bandit - Solo (feat. Demi Lovato)\nLil ...
...,...,...,...,...,...,...,...
28336,Matoma,Old Thing Back (feat. Ja Rule and Ralph Tresvant),6HTJZ0TQJVMSKkUGzAOe2h,69,True,2015,"Oh yeah, Bad Boy baby\nBiggie Smalls\nListen...\n\nUh yeah baby oooooooh, listen to me, oh yeaaa..."
28339,Pitbull,Timber (feat. Ke$ha),1zHlj4dQ8ZAtrayhuDDmkY,59,False,2013,"""Timber""\n(feat. Ke$ha)\n\n[Ke$ha]\nIt's going down, I'm yelling timber\nΧορευει και κατεβαινει ..."
28340,Robin Schulz,Headlights (feat. Ilsey),06Hdbxh6NCy6TIhjdXTchB,62,False,2015,1. Lean On - Major Lazer x DJ Snake feat. MØ\n\n2. Parijs - Kenny B\n3. Ain't Nobody (Loves Me B...
28341,Pitbull,Fun (feat. Chris Brown),6Zo3wgDE076K2AuiGZF3CS,61,False,2014,2005Whose Girl Is That 👕\nRun It! (feat. Juelz Santana) 👕\nYo (Excuse Me Miss) 👕\n2006Gimme That...


In [6]:
feat = []    
for values in df['title']:
    try:
        search = re.search(r'(?:\(|\[)(?:feat.|feat.|feat |with )[^)]*(?:\)|\])', values, re.I).group()
        feat.append(search.replace('(' or '[', '').replace('feat. ' or 'Feat. ' or 'with ', '').replace(')' or ']', '')) 

    except:
        feat.append('')
        
    
df['title'] = df['title'].str.replace(r'(?:\(|\[)(?:feat. |feat |with )[^)]*(?:\)|\])','', re.I)
df.insert(1, 'featuring', feat) 
#https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns?page=1&tab=votes#tab-top
# Thanks Brett for Regex Help

I'm just going to manually fix the rest of these, some didn't have brackets around them and for some reason they just aren't moving over.

In [7]:
df[df['title'].str.contains("feat")].head()

,artists,featuring,title,track_id,popularity,explicit,release_year,lyrics
18136,Beyoncé,,Video Phone - Extended Remix featuring Lady Gaga,7mzQFrKyyde7StPaXLss4K,34,False,2009,"[Intro: Beyoncé]\nUh, uh, uh, uh\nUh, uh, uh, uh\nUh, uh, uh, uh\nShawty, whatcho name is?\n\n[P..."
18238,Johnny Flynn,,The Water - feat. Laura Marling,69FraOspp1B24ZFBqlUaoV,37,False,2010,"[Verse 1]\nAll that I have is a river\nThe river is always my home\nLord, take me away, for I ju..."
19254,Britney Spears,,Slumber Party feat. Tinashe,2i9q8CteBmpslOJeQdMGqA,57,True,2016,Big Sean - Moves\nKris Kross Amsterdam & Conor Maynard - Are You Sure? feat. Ty Dolla $ign\nDJ S...
19636,Lil Yachty,,Oprah’s Bank Account (Lil Yachty & DaBaby feat. Drake),7wsmIIm0xWmtP7TmACXkJn,76,True,2020,Tones And I - Bad Child\nLil Yachty - Oprah’s Bank Account (Lil Yachty & DaBaby feat. Drake)\nJa...
20479,Craig David,,Rise & Fall - feat. Sting,7bZhjrvRcCfFkQE3iwxEIr,0,False,2002,[Chorus: Sting]\nSometimes in life you feel the fight is over\nAnd it seems as though the writin...


Manually fixing the remaining rows where feat. and with are still present

In [8]:
def feat_title(track_id, feat, title):
    df.loc[df['track_id'] == track_id, ['featuring', 'title']] = [feat,title]

In [9]:
feat_title('7mzQFrKyyde7StPaXLss4K', 'Lady Gaga', 'Video Phone')
feat_title('69FraOspp1B24ZFBqlUaoV', 'Laura Marling', 'The Water')
feat_title('2i9q8CteBmpslOJeQdMGqA', 'Tinashe', 'Slumber Party')
feat_title('7wsmIIm0xWmtP7TmACXkJn', 'Lil Yachty, DaBaby & Drake', 'Oprah’s Bank Account')
feat_title('7bZhjrvRcCfFkQE3iwxEIr', 'Sting', 'Rise & Fall')
feat_title('6s9m5J92By7jii22Q2XtY2', 'Carrie Underwood', "Somethin' Bad")
feat_title('4CyZpcjZdbAAD0J0EPSwmF', 'Take 4', 'Idiot Wind')
feat_title('3oQA7hcoD7zArMPLrADBts', 'Sune Rose Wagner', 'Deceive')
feat_title('1dWoElKx73yU6u3xUPjFIe', 'Low', 'The Dream')
feat_title('2jPXk7okWWF8AYkz2tNy6o', 'Ghost Society', 'River Of LIfe')
feat_title('6TqDrmjUDqQo4JiF1apQnL', 'Summertime Sadness', 'Keelan Donovan')
feat_title('2iNnrc56yZP4jBBLkPc5bx', 'Matt Wright', 'Take Me To Church')
feat_title('sqhKYHmQSGmGWzrgyL2nc', 'Mumford & Sons and Paul Simon', 'The Boxer')
feat_title('6pPD1k342KK4fMU6xPHufi', 'Avenue Beat', 'Watermelon Sugar')
feat_title('28Q2F391hseaFE88B3jki4', 'Cedella Marley', "Good Ol' Days")
feat_title('24MWSatP5sTylISvJONhnB', 'Justin Bieber', "Falling For You")
feat_title('04LAPXr5C6W4ufoCVxr5Bx', 'Christina Aguilera', "Steppin' Out with My Baby")

Making sure there are no spaces at beginning and end

In [10]:
df['artists'] = df['artists'].str.strip()
df['featuring'] = df['featuring'].str.strip()
df['title'] = df['title'].str.strip()

### Re-Filling Missing / Misfilled lyrics

In [11]:
def get_lyrics():
    genius = lyricsgenius.Genius("2AaY5B5Iylvf3X2oQl_4Jmk74pSZ-J8VqcboMk1PO_y7N8wlN83D9a85uhNh49Id")
    genius.timeout = 15
    for track in df.values:
        if track[1] != '':
            retries = 0
            while retries < 3:
                try:
                    song = genius.search_song(track[2], track[0])
                    if song is not None:
                        df.loc[df['featuring'] == track[1], 'lyrics'] = song.lyrics
                    else:
                        df.loc[df['featuring'] == track[1], 'lyrics'] = np.NaN
                    break
                except Timeout as e:
                    retries += 1
                    continue
        else:
            pass

In [12]:
get_lyrics()

Searching for "Video Phone" by Beyoncé...
Done.
Searching for "The Water" by Johnny Flynn...
Done.
Searching for "Way Too Pretty for Prison" by Miranda Lambert...
Done.
Searching for "Somethin' Bad" by Miranda Lambert...
Done.
Searching for "Smokin' and Drinkin'" by Miranda Lambert...
Done.
Searching for "Ordinary World" by Green Day...
Done.
Searching for "Start Again" by OneRepublic...
Done.
Searching for "Rich Love" by OneRepublic...
Done.
Searching for "The Fighter" by Gym Class Heroes...
Done.
Searching for "Nothing Compares 2 U  - Live" by Prince...
Done.
Searching for "Rain" by Rag'n'Bone Man...
Done.
Searching for "Hell Yeah" by Rag'n'Bone Man...
Done.
Searching for "How Come, How Long" by Babyface...
Done.
Searching for "Over Now" by Calvin Harris...
Done.
Searching for "Pray For Me" by The Weeknd...
Done.
Searching for "Power is Power" by SZA...
Done.
Searching for "Or Nah  - Remix" by Ty Dolla $ign...
Done.
Searching for "You Can't Stop Me" by Brett Eldredge...
Done.
Searchi

Done.
Searching for "Intervention" by Margaret Cho...
Done.
Searching for "Calling in Stoned" by Margaret Cho...
Done.
Searching for "Your Dick" by Margaret Cho...
Done.
Searching for "Baby I'm with the Band" by Margaret Cho...
Done.
Searching for "Hey Big Dog" by Margaret Cho...
Done.
Searching for "I'm Sorry" by Margaret Cho...
Done.
Searching for "Skechers" by DripReport...
Done.
Searching for "Hermosa" by Aastha Gill...
Done.
Searching for "Garmi (From "Street Dancer 3D")" by Badshah...
Done.
Searching for "Hey Mama" by David Guetta...
Done.
Searching for "You Are My Sunshine" by Morgane Stapleton...
Done.
Searching for "Every Breath You Take" by Aaron Krause...
Done.
Searching for "Everything Is Free" by Sylvan Esso...
Done.
Searching for "My Happiness" by John Prine...
Done.
Searching for "Just Breathe" by Willie Nelson...
Done.
Searching for "Color of the Blues" by John Prine...
Done.
Searching for "Both Sides Now" by Alva Leigh...
Done.
Searching for "Water" by The Jepettos...


Done.
Searching for "Black" by Buddy...
Done.
Searching for "Freedom" by Beyoncé...
Done.
Searching for "Mr. Officer" by Tee Grizzley...
Done.
Searching for "S.O.T.L. (Silence of the Lambs)" by Ludacris...
Done.
Searching for "Mad" by Solange...
Done.
Searching for "I JUST WANNA LIVE" by Keedron Bryant...
Done.
Searching for "Crew" by GoldLink...
Done.
Searching for "Sad" by Kodie Shane...
Done.
Searching for "Wanna Know Remix" by Dave...
Done.
Searching for "NozeBleedz" by Ray jr....
Done.
Searching for "Grove" by Elysia Crampton...
Done.
Searching for "We Are One" by Barack Obama...
Done.
Searching for "Day That I Die" by Zac Brown Band...
Done.
Searching for "The Other Boys" by NERVO...
Done.
Searching for "Gangsta's Paradise" by Coolio...
Done.
Searching for "Pearl Cadillac" by Gary Clark Jr....
Done.
Searching for "Rain Dance" by Hollis Brown...
Done.
Searching for "Promise Land" by Hannah Miller...
Done.
Searching for "Old Town Road  - Remix" by Lil Nas X...
Done.
Searching for "

This eliminated 800 missing or duplicated issues.

In [141]:
pd.set_option('display.max_rows', 10)

In [73]:
df.drop(df.loc[df['lyrics'] == '[Instrumental]'].index, inplace=True) # confirmed on genius site, the song lyrics are just 'Instrumental'
df.drop(df.loc[df['lyrics'] == '(Instrumental)'].index, inplace=True)
df.drop(df.loc[df['artists'] == 'Steve Martin'].index, inplace=True)
df.drop(df.loc[df['artists'] == 'Various Artists'].index, inplace=True) # can't look up songs by various artists
df.drop(df.loc[df['artists'] == 'Monty Python'].index, inplace=True)
df.drop(df.loc[df['artists'] == 'Shane McAnally'].index, inplace=True) # All were interviews and the few that were songs genius didn't have lyrics



In [74]:
dups = df[df.duplicated('lyrics', keep = False)].sort_values('lyrics')

In [75]:
dups.shape

(2086, 8)

In [76]:
dups['title'] = dups['title'].str.replace(r'-[^-]*$','', re.I)

In [77]:
def get_lyrics():
    genius = lyricsgenius.Genius("2AaY5B5Iylvf3X2oQl_4Jmk74pSZ-J8VqcboMk1PO_y7N8wlN83D9a85uhNh49Id")
    genius.timeout = 15
    for track in df.values:
        for dup in dups.values:
            if track[3] == dup[3] :
                retries = 0
                while retries < 3:
                    try:
                        song = genius.search_song(dup[2], dup[0])
                        if song is not None:
                            df.loc[df['track_id'] == track[3], 'lyrics'] = song.lyrics
                        else:
                            df.loc[df['track_id'] == track[3], 'lyrics'] = np.NaN
                        break
                    except Timeout as e:
                        retries += 1
                        continue
            else:
                pass

In [78]:
get_lyrics()

Searching for "Wild Fire " by Laura Marling...
Done.
Searching for "Kathy’s Song " by Laura Marling...
Done.
Searching for "The Water" by Johnny Flynn...
Done.
Searching for "Old Sh!t" by Miranda Lambert...
No results found for: 'Old Sh!t Miranda Lambert'
Searching for "Audience Observation / Dope Song / Band Names Rap " by Neil Young...
Done.
Searching for "Be My Husband " by Nina Simone...
Done.
Searching for "Sinnerman " by Nina Simone...
Done.
Searching for "Black Is The Color Of My True Love's Hair " by Nina Simone...
Done.
Searching for "Baby Beluga " by Raffi...
Done.
Searching for "Baby Beluga" by Raffi...
Done.
Searching for "6 Little Ducks" by Raffi...
Done.
Searching for "Young People Marching (for Greta Thunberg)" by Raffi...
Done.
Searching for "Grace (We All Try)" by Rag'n'Bone Man...
Done.
Searching for "Grace" by Rag'n'Bone Man...
Done.
Searching for "Remains (Bastille Vs. Rag’N’Bone Man Vs. Skunk Anansie)" by Bastille...
No results found for: 'Remains (Bastille Vs. Rag

No results found for: 'Mule Skinner Blues  The Wildwood Boys'
Searching for "Deep Elem Blues " by Jerry Garcia Acoustic Band...
No results found for: 'Deep Elem Blues  Jerry Garcia Acoustic Band'
Searching for "Spike Driver Blues " by Jerry Garcia Acoustic Band...
No results found for: 'Spike Driver Blues  Jerry Garcia Acoustic Band'
Searching for "Ragged but Right " by Jerry Garcia Acoustic Band...
No results found for: 'Ragged but Right  Jerry Garcia Acoustic Band'
Searching for "Sittin' Here In Limbo" by Jerry Garcia...
Done.
Searching for "Bright Morning Star " by Jerry Garcia Acoustic Band...
No results found for: 'Bright Morning Star  Jerry Garcia Acoustic Band'
Searching for "The Sweet Sunny South" by Jerry Garcia...
No results found for: 'The Sweet Sunny South Jerry Garcia'
Searching for "Grateful Dawg" by Jerry Garcia...
No results found for: 'Grateful Dawg Jerry Garcia'
Searching for "Sally Goodin " by Sleepy Hollow Hog Stompers...
Done.
Searching for "Long Black Veil" by Jer

Done.
Searching for "Norwegian Wood (Spanish Wood)" by David Axelrod...
No results found for: 'Norwegian Wood (Spanish Wood) David Axelrod'
Searching for "Suite bergamasque: Suite bergamasque: III. Clair de lune. Andante très expressif" by Claude Debussy...
No results found for: 'Suite bergamasque: Suite bergamasque: III. Clair de lune. Andante très expressif Claude Debussy'
Searching for "Ständchen in D Minor (After Schubert), S. 560" by Franz Liszt...
No results found for: 'Ständchen in D Minor (After Schubert), S. 560 Franz Liszt'
Searching for "Nocturne en mi bémol majeur opus 9 n°2: Ballade en Sol Mineur No.1" by Frédéric Chopin...
No results found for: 'Nocturne en mi bémol majeur opus 9 n°2: Ballade en Sol Mineur No.1 Frédéric Chopin'
Searching for "Serenade for Strings in E Major, Op. 22, B. 52: II. Tempo di valse" by Antonín Dvořák...
No results found for: 'Serenade for Strings in E Major, Op. 22, B. 52: II. Tempo di valse Antonín Dvořák'
Searching for "Vesper" by New Tide Orq

Done.
Searching for "A Village Romeo and Juliet: The Walk to the Paradise Garden" by Frederick Delius...
No results found for: 'A Village Romeo and Juliet: The Walk to the Paradise Garden Frederick Delius'
Searching for "Piano Trio No. 2 in F Major, Op. 80: Piano Trio No. 2 in F Major, Op. 80: III. In mäßiger Bewegung" by Robert Schumann...
No results found for: 'Piano Trio No. 2 in F Major, Op. 80: Piano Trio No. 2 in F Major, Op. 80: III. In mäßiger Bewegung Robert Schumann'
Searching for "Trio No. 3 en ut mineur, Op. 101: III. Andante grazioso" by Johannes Brahms...
No results found for: 'Trio No. 3 en ut mineur, Op. 101: III. Andante grazioso Johannes Brahms'
Searching for "Wein, Weib und Gesang! (Wine, Woman and Song!), Op. 333" by Johann Strauss II...
No results found for: 'Wein, Weib und Gesang! (Wine, Woman and Song!), Op. 333 Johann Strauss II'
Searching for "6 Moments musicaux, Op.16: No. 5 in D flat, Adagio sostenuto" by Sergei Rachmaninoff...
No results found for: '6 Moment

No results found for: 'Pieces de clavecin, Book 1: I. L'Aimable: Gracieux Joseph-Nicolas-Pancrace Royer'
Searching for "Dernière courente de Gautier" by Denis Gaultier...
No results found for: 'Dernière courente de Gautier Denis Gaultier'
Searching for "Partita for Violin Solo No.2 in D minor, BWV 1004: 5. Ciaccona" by Johann Sebastian Bach...
No results found for: 'Partita for Violin Solo No.2 in D minor, BWV 1004: 5. Ciaccona Johann Sebastian Bach'
Searching for "Membra Jesu nostri, BuxWV 75, Cantata V "Ad pectus": II. Concerto I. Sicut modo geniti infantes" by Dietrich Buxtehude...
No results found for: 'Membra Jesu nostri, BuxWV 75, Cantata V "Ad pectus": II. Concerto I. Sicut modo geniti infantes Dietrich Buxtehude'
Searching for "Concerto for Flute and Violin in E Minor, TWV 52:e3: Concerto for Flute and Violin in E Minor, TWV 52:e3: I. No indication" by Georg Philipp Telemann...
No results found for: 'Concerto for Flute and Violin in E Minor, TWV 52:e3: Concerto for Flute and Vi

...

No results found for: 'Sibelius : Finlandia Op.26 Jean Sibelius'
Searching for "Merry Christmas Mr. Lawrence (Arr. Metcalfe)" by Jess Gillam...
No results found for: 'Merry Christmas Mr. Lawrence (Arr. Metcalfe) Jess Gillam'
Searching for "Hungarian Dance No.5 in G Minor, WoO 1, No.5" by Johannes Brahms...
No results found for: 'Hungarian Dance No.5 in G Minor, WoO 1, No.5 Johannes Brahms'
Searching for "Piano Concerto No. 5 in E-Flat Major, Op. 73 "Emperor": 2. Adagio un poco mosso " by Ludwig van Beethoven...
No results found for: 'Piano Concerto No. 5 in E-Flat Major, Op. 73 "Emperor": 2. Adagio un poco mosso  Ludwig van Beethoven'
Searching for "Symphony No.5 in E flat, Op.82: 3. Allegro molto" by Jean Sibelius...
No results found for: 'Symphony No.5 in E flat, Op.82: 3. Allegro molto Jean Sibelius'
Searching for "Pini di Roma (The Pines of Rome): III. I pini del Gianiclo" by Ottorino Respighi...
No results found for: 'Pini di Roma (The Pines of Rome): III. I pini del Gianiclo Otto

No results found for: 'Songs from "Friday Afternoons", Op.7: Songs From Friday Afternoons, Op. 7: "Cuckoo!" Benjamin Britten'
Searching for "The Firebird (L'oiseau De Feu) " by Igor Stravinsky...
Done.
Searching for "Holberg Suite, Op. 40: 1. Präludium (Allegro vivace)" by Edvard Grieg...
No results found for: 'Holberg Suite, Op. 40: 1. Präludium (Allegro vivace) Edvard Grieg'
Searching for "Dirty River" by Husky...
Specified song does not contain lyrics. Rejecting.
Searching for "Fire & Fury " by Overcoats...
Done.
Searching for "Almost Mine (Acoustic)" by Anthony da Costa...
Done.
Searching for "Heavenly Track" by Kaia Kater...
Done.
Searching for "The Time Has Come" by John Smith...
Done.
Searching for "The Draft Horse" by Jeff Ingersoll...
No results found for: 'The Draft Horse Jeff Ingersoll'
Searching for "Glow (Acoustic)" by Brooke Annibale...
No results found for: 'Glow (Acoustic) Brooke Annibale'
Searching for "Oh, How It Rained" by Gabriel Kelley...
Done.
Searching for "Love 

No results found for: 'The Unquiet Grave Siobhan Miller'
Searching for "Tu n'existes pas" by Francesca Blanchard...
No results found for: 'Tu n'existes pas Francesca Blanchard'
Searching for "Que Facil Es Hablar" by Karina Vismara...
No results found for: 'Que Facil Es Hablar Karina Vismara'
Searching for "Californie" by Jeffrey Piton...
Specified song does not contain lyrics. Rejecting.
Searching for "Yimenda" by Emily Wurramara...
No results found for: 'Yimenda Emily Wurramara'
Searching for "Bússola" by Duda In The Sky...
No results found for: 'Bússola Duda In The Sky'
Searching for "'S Ar Maidin Moch is Mé ar mo Leabain Bhoig" by Muireann Nic Amhlaoibh...
No results found for: ''S Ar Maidin Moch is Mé ar mo Leabain Bhoig Muireann Nic Amhlaoibh'
Searching for "Amwaj (Waves)" by Amira Kheir...
No results found for: 'Amwaj (Waves) Amira Kheir'
Searching for "Amor Papaya en Invierno" by Carlos Sadness...
No results found for: 'Amor Papaya en Invierno Carlos Sadness'
Searching for "Peu 

Done.
Searching for "The Best " by James Bay...
Done.
Searching for "Personal Jesus " by Def Leppard...
Done.
Searching for "Give Yourself a Try " by The Hunna...
Done.
Searching for "Running Up That Hill " by First Aid Kit...
Done.
Searching for "A Whiter Shade Of Pale " by Rhye...
Done.
Searching for "For You " by Seinabo Sey...
Done.
Searching for "Tougher Than The Rest " by Midland...
Done.
Searching for "Walking on Broken Glass " by Lake Street Dive...
Done.
Searching for "Time Fades Away " by Yo La Tengo...
Done.
Searching for "One Kiss " by Scarlet Pleasure...
Done.
Searching for "Blue Canadian Rockies " by Ray LaMontagne...
Done.
Searching for "Khuda Bhi Aasman " by Khruangbin...
Done.
Searching for "Leave (Get Out) " by Anne-Marie...
Done.
Searching for "Passionfruit " by Cornelius...
Done.
Searching for "Juana Mecho " by Vicente Garcia...
Done.
Searching for "Obsession " by Hillsong Worship...
Done.
Searching for "Forever Cold" by Smooth Hound Smith...
Done.
Searching for "Li

Done.
Searching for "An Dòrd Fienne (Òrò Sè Do Bheatha Abhaile)" by The Wolfe Tones...
No results found for: 'An Dòrd Fienne (Òrò Sè Do Bheatha Abhaile) The Wolfe Tones'
Searching for "The Galtee Mountain Boy" by Paddy Reilly...
Done.
Searching for "He Moved Through The Fair" by Sinéad O'Connor...
Done.
Searching for "Only Our Rivers Run Free" by Paddy Reilly...
Done.
Searching for "Will Ye Go Lassie Go" by Jim McCann...
No results found for: 'Will Ye Go Lassie Go Jim McCann'
Searching for "Michael " by Johnny McEvoy...
Done.
Searching for "The Jail of Cluain Meala" by The Dubliners...
No results found for: 'The Jail of Cluain Meala The Dubliners'
Searching for "Erin-Go" by Kate Burke & Ruth Hazleton...
No results found for: 'Erin-Go Kate Burke & Ruth Hazleton'
Searching for "Boolavogue" by Brendan Bowyer...
No results found for: 'Boolavogue Brendan Bowyer'
Searching for "Dicey Riley" by Dominic Behan...
No results found for: 'Dicey Riley Dominic Behan'
Searching for "The Man from the 

In [83]:
df[df.duplicated('lyrics', keep = False)]

,artists,featuring,title,track_id,popularity,explicit,release_year,lyrics
18211,Laura Marling,,Wild Fire - Mahogany Sessions,0ObkEn7HhjHWCdoyv0ZeHA,0,False,2018,[Verse 1]\nChasing down a wild fire\nAre you trying to make a cold liar out of me?\nYou want to ...
18213,Laura Marling,,Wild Fire - Recorded at Spotify Studios NYC,4aWsyx9WR3xQOQtoHXOX3t,36,True,2017,[Verse 1]\nChasing down a wild fire\nAre you trying to make a cold liar out of me?\nYou want to ...
18216,Laura Marling,,Kathy’s Song - Recorded at Spotify Studios NYC,7nebNowPcMMBKgjaS1nJlm,43,False,2017,YONAKA\nRockstar\nYou Need To Calm Down\n\nDennis Lloyd\nAura\nLike a Stone\n\nDenzel Curry\nCLO...
18238,Johnny Flynn,Laura Marling,The Water,69FraOspp1B24ZFBqlUaoV,37,False,2010,"[Verse 1]\nAll that I have is a river\nThe river is always my home\nLord, take me away, for I ju..."
18284,Miranda Lambert,,Old Sh!t,7nRl82OQNxuZRQmg2T7OYN,40,False,2014,NaN
18386,Neil Young,,Audience Observation / Dope Song / Band Names Rap - Live at the Riverboat 1969,4bACPr2J3ayQEQgiKvvDzn,36,False,2009,CHAPTER ONE\n\nNAMES OF PEOPLE: THE DUCHESSE DE GUERMANT...
18396,Nina Simone,,Be My Husband - Live In New York/1965,1uSedrh9ywZJKQje4RWi7d,0,False,1965,Be my husband man I be your wife\nBe my husband man I be your wife\nBe my husband man I be your ...
18408,Nina Simone,,Sinnerman - Live In New York/1965,5XI9hva1F1WCRSvNNYCK1w,0,False,1965,"Oh, Sinnerman, where you gonna run to?\nSinnerman, where you gonna run to?\nWhere you gonna run ..."
18412,Nina Simone,,Black Is The Color Of My True Love's Hair - Live In New York/1964,1KBCVBuIeC60AqIOad0pJD,0,False,1966,Black is the color of my true love's hair\nHis face so soft and wondrous fair\nThe purest eyes\n...
18560,Raffi,,Baby Beluga - 40th Anniversary Version,2JTG11HiYHJc4hu5LTDry5,37,False,2020,Baby Beluga in the deep blue sea\nSwim so wild and swim so free\nHeaven above and the sea below\...


I started with this part of the dataset that had 3166 values in the lyrics marked as 'duplicate' where 1611 were null.
Unfortunately because of the way they are retrieved with the API, they aren't all true duplicates, so I could not just drop them except the first occurrence. Ideally when a lyric was not found for the track my code directed it to append NaN to the column value for that track. However the API made it seem like there were lyrics and appended text from the page.
<p> Removing (feat. and with) in the title and retrieving the lyrics for the rows with those changes eliminated 866 NaN / 'duplicated' rows. I now had 2086 rows to take care of. I noticed that some titles that had -Remastered -Live or some variation of that were also not pulled properly. Once I removed the '-characters' within the duplicated list dataframe and reran the fetch. I was down to 1942 'dups' with 1089 'NaNs' </p>
<br>For the remaining values I glanced through and comparing to Genius, manually filled the values in with a function. Of the 1741 'duplicates', 81 of them were true duplicates and 45 that I could fill in with research. </br>
The remainder of these sets were dropped for various reasons: Instrumental only, classical music, comedy skits, movie skits, lyrics did not exist on Genius, etc.


In [84]:
df.drop([18560,18396, 18142, 24055, 26249, 26340, 21250, 19292, 23153, 23015, 22335, 27349, 20730, 26269, 27047, 23754, 18408, 23305, 25900, 22748, 26535, 24798, 19535, 23311, 26047, 19283, 20975, 23008, 25515, 26945, 26928, 27289, 20338, 20875, 19101, 19320, 25317, 20617, 20507, 18885, 24323, 23382, 19584, 19872, 23693, 19011, 25536, 26539, 18238, 26941, 21254, 23703, 24029, 23987, 19540, 19522, 19574, 20777, 20453, 18882, 26938, 25295, 19037, 19509, 19891, 23076, 23315, 20335, 25282, 18679, 19044, 19030, 19045, 23005, 19585, 26284, 25897, 26292, 18412, 23738, 25934], inplace=True)

In [94]:
def fill_lyrics(artist, title, track_id):
    genius = lyricsgenius.Genius("2AaY5B5Iylvf3X2oQl_4Jmk74pSZ-J8VqcboMk1PO_y7N8wlN83D9a85uhNh49Id")
    song = genius.search_song(title, artist)
    df.loc[df['track_id'] == track_id, 'lyrics'] = song.lyrics
    

In [95]:
fill_lyrics('Laura Marling', 'Wild Fire', '4aWsyx9WR3xQOQtoHXOX3t')
fill_lyrics('Chuck Berry', "Sweet Little Rock 'N' Roller", '2xhqH3KUOTeU4siaNQnerm')
fill_lyrics('The Civil Wars', "Dance Me to the End of Lov", '6tZAbv5JEsfqjTpkBOrLje')
fill_lyrics('The Beach Boys', "All Summer Long", '6NIhm22UWaECPEdGm7VqEC')
fill_lyrics('JONES', "Melt", '6W83GER4shtT9lpkYzEYeq')
fill_lyrics('Julia Michaels', 'Give It To You', '0U2HEoCazR0hZlNavNeWov')
fill_lyrics('Henry Jamison', "Through a Glass", '5WtvoPeSpp6pIp9bG6arVS')
fill_lyrics('Chuck Berry', "Thirty Days", '4mOF1z5HEmaFyaWkyYKxC6q')
fill_lyrics('Creature Comfort', 'Big, Buff, and Handsome', '50yOmaugZveFg8EpcsR1yh')
fill_lyrics('Josh Abbott Band', "Wasn't That Drunk", '6yop3isbCrce4maIoUdH0F')
fill_lyrics('Susto', 'Weather Balloons', '4mOmMccRWthpaUtjPhiQm8')
fill_lyrics('Birdy', 'The A Team', '0qSSbcZ2bvjJuB3Wy5lzK5')
fill_lyrics('Toots & The Maytals', 'Take Me Home, Country Road', '0iAJ5J7hGRnaY3zWXACbws')
fill_lyrics('Susto', 'Weather Balloons', '4mOmMccRWthpaUtjPhiQm8')
fill_lyrics('Pavement', 'The Classical', '4ry1iCeZDOKK9NFtmXDFsH')
fill_lyrics('Bastille', 'Remains', '6G43t8BCjmJyPGwNrAAPPV')
fill_lyrics('Justin Timberlake', "CAN'T STOP THE FEELING!", '6JV2JOEocMgcZxYSZelKcc')
fill_lyrics('Tony Bennett', 'The Shadow of Your Smile', '66YI4JFp7BcMcepibGprsN') 
fill_lyrics('trentemøller', 'Hands Down', '7ie4WcGBbqVS1rB0TLzWD3')
fill_lyrics('trentemøller', 'One Eye Open', '0uKONW34B7aWwT4RjPZyBS')
fill_lyrics('The Fall','Container Drivers', '3qaggCdofFfdiHukkHIRFx')
fill_lyrics('The Undertones','When Saturday Comes', '3Wflvjj1hHaXnhjhByFOhW')
fill_lyrics('Orchestral Manoeuvres In The Dark', "Of All The Things We've Made", '2EmI9kp6hoA3Iwd3SDUbk4')
fill_lyrics('Urusei Yatsura','First Day on a New Plane', '7kqDFDhY0cpihNgP7R7J61')
fill_lyrics('Bee Gees', 'Massachusetts', '1mBPlknQe00PCd7R9g34ix')
fill_lyrics('T. Rex','Salamanda Palaganda', '2SvqcWmKhJyPuPD7yudz9k')
fill_lyrics('Anna Kendrick', 'Cups', '2sCl0FFOzZSmYZs90zOpd1')
fill_lyrics('Golden Youth',"Where's Your Heart Gone", '2nNgB4HV5Q1MugwVFMUPZZ')
fill_lyrics('Kacey Musgraves', 'Apologize', '6QT53ohGZeLGoYktBPRePD')
fill_lyrics('Vanessa Carlton','A Thousand Miles', '6t9smWoGTgUF2LVSQuJK5P')
fill_lyrics('Kishi Bashi', 'This Must Be the Place', '0udJrKQOgJz6TwZXuhi7IM')
fill_lyrics('Beta Radio','Kilimanjaro', '6HLYA51RLvwRKEtpEfjQHl')

fill_lyrics('Sufjan Stevens', 'Mystery of Love', '0oTtnnedK0C4unALxVTPhz')
fill_lyrics('Xavier Omär', 'Blind Man', '1dx7c3AyA4F9m82gXMqSQU')
fill_lyrics('M. Ward', 'Never Had Nobody Like You', '7LT6dkoxfUavcBDe7RYgog')
fill_lyrics('Peter Sarstedt','Where Do You Go To (My Lovely)','1D0ysB6OY7LsXxrjN1Zvb2')


fill_lyrics('Leif Vollebekk', 'Rest', '54zNVHj8N3wjCb4UpqSvKw')
fill_lyrics('Charlie Cunningham', 'Lights Off', '34bvDVeoJZyiuxfLUmNzwV')

fill_lyrics('Jesca Hoop', 'Tulip', '0kmVl9LVSmzQ7ssF9FnCWZ')
fill_lyrics('Noah Reid','Simply the Best','1KFuLRUmWf0irn0CLclT2X')
fill_lyrics('High Highs','Catch the Wind','6HMHm0lHF1pe5hFsEgcZ3V')
fill_lyrics('Roo Panes', 'Lullaby Love', '6bamFKyxmUKFQYuiYoutWS')
fill_lyrics('New World Punx', 'Memories', '6R4c9KUUlb16SaanafvX4n')
fill_lyrics('Laidback Luke' , 'Collide', '4XEM2IBeOp1W0USZXi2SGC')
fill_lyrics('Lana Del Rey','Summertime Sadness', 'PUIzlqotEmPuBfjbwYWOB')

Searching for "Wild Fire" by Laura Marling...
Done.
Searching for "Sweet Little Rock 'N' Roller" by Chuck Berry...
Done.
Searching for "Dance Me to the End of Lov" by The Civil Wars...
Done.
Searching for "All Summer Long" by The Beach Boys...
Done.
Searching for "Melt" by JONES...
Done.
Searching for "Give It To You" by Julia Michaels...
Done.
Searching for "Through a Glass" by Henry Jamison...
Done.
Searching for "Thirty Days" by Chuck Berry...
Done.
Searching for "Big, Buff, and Handsome" by Creature Comfort...
Done.
Searching for "Wasn't That Drunk" by Josh Abbott Band...
Done.
Searching for "Weather Balloons" by Susto...
Done.
Searching for "The A Team" by Birdy...
Done.
Searching for "Take Me Home, Country Road" by Toots & The Maytals...
Done.
Searching for "Weather Balloons" by Susto...
Done.
Searching for "The Classical" by Pavement...
Done.
Searching for "Remains" by Bastille...
Done.
Searching for "CAN'T STOP THE FEELING!" by Justin Timberlake...
Done.
Searching for "The Shad

In [97]:
# df.to_csv("./data/lyrics_cp2clean.csv", index=False)

In [157]:
df = pd.read_csv('./data/lyrics_cp2clean.csv')
df.shape

(9449, 8)

### Dropping Null Lyrics
Since this is an NLP project, if the lyrics column have not gathered any values (after running through the pull request twice), it is time to drop these songs. There may be some 'duplicates' based on lyrics, I will keep those, because it is the artist who decides to sing their own name, I may listen to the song from one artist, but not the other.

In [158]:
df.drop(df[df.duplicated('lyrics', keep = False)].index, inplace = True)

In [159]:
df

,artists,featuring,title,track_id,popularity,explicit,release_year,lyrics
0,Lady Gaga,NaN,The Edge Of Glory,57F80059mWaWFvWG3tBTbO,66,False,2011,"[Verse 1]\nThere ain't no reason you and me should be alone\nTonight, yeah, baby\nTonight, yeah,..."
1,Lady Gaga,NaN,Million Reasons,1rpDLMm4FjGqMz6N1dZZo8,54,False,2016,[Verse 1]\nYou're giving me a million reasons to let you go\nYou're giving me a million reasons ...
2,Lady Gaga,NaN,You And I,1ADciGUxOaUbBFd0rpYx7d,62,False,2011,"[Verse 1: Lady Gaga]\nIt’s been a long time since I came around\nBeen a long time, but I’m back ..."
3,Lady Gaga,NaN,Heal Me,5Ctq1DOZUWZqsQZXro2jf7,61,False,2018,[Verse 1]\nTreat me like your patient\nJust don't keep me waiting\nOr I'll just be wasted\nIn a ...
4,Lady Gaga,NaN,John Wayne,1vwTGFOq34pA2d0VUikL3W,47,False,2016,"[Intro: Spoken]\nIt's like, I just love a cowboy\nYou know\nI'm just like, I just, I know, it's ..."
...,...,...,...,...,...,...,...,...
9444,Amy Winehouse,NaN,Tears Dry On Their Own,1irwMHD7nuvuZulBpzATv1,42,True,2006,[Produced by Salaam Remi]\n\n[Verse 1]\nAll I can ever be to you is the darkness that we knew\nA...
9445,Mos Def,NaN,Mathematics,3gRlmtdCyNoKiyozn2pqc9,60,True,1999,"[Produced by DJ Premier]\n\n[Intro]\nBucka-bucka-bucka-bucka-bucka-bucka, haha!\nYou know the de..."
9446,Amy Winehouse,NaN,Stronger Than Me,5LC7nItIEFp4nzdFdEGbf9,64,False,2015,[Verse 1]\nYou should be stronger than me\nYou been here seven years longer than me\nDon't you k...
9447,Mary J. Blige,NaN,No More Drama,7nWsh14AHpnTzq1Xejz90U,53,False,2001,"[Intro]\nSo tired, tired of this drama\nYou go your way, I go my way\nNo more, no more\nI wanna ..."


### Cleaning up the Lyrics
The lyrics column has some obscure character such as \n and \ within the strings that need to be removed.
<br> The lyrics also contain [Verse] sections that list which artist is singing that part. This needs to be removed because it isn't part of the song and it will throw my models off, when it looks for the artist in the lyrics and creating my skip column.</br>

In [161]:
df['lyrics'] = df['lyrics'].str.replace('\n', '').str.replace("\'", "")

In [163]:
df['lyrics'] = df['lyrics'].str.replace(r'[\(\[].*?[\)\]]','')

In [164]:
df

,artists,featuring,title,track_id,popularity,explicit,release_year,lyrics
0,Lady Gaga,NaN,The Edge Of Glory,57F80059mWaWFvWG3tBTbO,66,False,2011,"There aint no reason you and me should be aloneTonight, yeah, babyTonight, yeah, babyBut I got a..."
1,Lady Gaga,NaN,Million Reasons,1rpDLMm4FjGqMz6N1dZZo8,54,False,2016,Youre giving me a million reasons to let you goYoure giving me a million reasons to quit the sho...
2,Lady Gaga,NaN,You And I,1ADciGUxOaUbBFd0rpYx7d,62,False,2011,"It’s been a long time since I came aroundBeen a long time, but I’m back in townThis time I’m not..."
3,Lady Gaga,NaN,Heal Me,5Ctq1DOZUWZqsQZXro2jf7,61,False,2018,Treat me like your patientJust dont keep me waitingOr Ill just be wastedIn a crowd of the lonely...
4,Lady Gaga,NaN,John Wayne,1vwTGFOq34pA2d0VUikL3W,47,False,2016,"Its like, I just love a cowboyYou knowIm just like, I just, I know, its badBut Im just likeCan I..."
...,...,...,...,...,...,...,...,...
9444,Amy Winehouse,NaN,Tears Dry On Their Own,1irwMHD7nuvuZulBpzATv1,42,True,2006,All I can ever be to you is the darkness that we knewAnd this regret Ive got accustomed toOnce i...
9445,Mos Def,NaN,Mathematics,3gRlmtdCyNoKiyozn2pqc9,60,True,1999,"Bucka-bucka-bucka-bucka-bucka-bucka, haha!You know the deal; its just me, yoBeats by Su-Primo fo..."
9446,Amy Winehouse,NaN,Stronger Than Me,5LC7nItIEFp4nzdFdEGbf9,64,False,2015,You should be stronger than meYou been here seven years longer than meDont you know you supposed...
9447,Mary J. Blige,NaN,No More Drama,7nWsh14AHpnTzq1Xejz90U,53,False,2001,"So tired, tired of this dramaYou go your way, I go my wayNo more, no moreI wanna be freeIm so ti..."


In [180]:
skip = []
for row in df.values:
    if row[7] in (row[0]) or row[7] in (row[2]):
        skip.append(1)
    else:
        skip.append(0)

In [189]:
df['skip'] = skip

In [191]:
df['skip'].value_counts()

0    7689
1       7
Name: skip, dtype: int64

In [188]:
df[df.duplicated('lyrics', keep = False)]

,artists,featuring,title,track_id,popularity,explicit,release_year,lyrics
365,One Direction,NaN,One Thing,5G2c6FsfTzgYUzageCmfXY,74,False,2012,Ive tried playing it coolBut when Im looking at youI cant ever be braveCause you make my heart r...
399,One Direction,NaN,One Thing - Acoustic,25wYrlnhv7bodDFsNajNEl,39,False,2012,Ive tried playing it coolBut when Im looking at youI cant ever be braveCause you make my heart r...
732,The Beach Boys,NaN,Our Prayer,1IysBwoeG9FSWCY8qLNqzL,42,False,2011,
2822,Mogwai,NaN,Hunted By A Freak,41VZ0TAbebnmk2Y2mkypas,0,False,2005,
2868,David Axelrod,NaN,The Mental Traveler,0KAXs9NI835prXCEvC7Mwg,17,False,2005,♫ ♫
...,...,...,...,...,...,...,...,...
4644,Jake Blount,NaN,"Goodbye, Honey, You Call That Gone",74KdVnlCyEqUod4bti2GBc,43,False,2020,
6410,Franz Liszt,NaN,Les Preludes,3oYwpPMHASKBUe7DDa7cVF,26,False,1992,♫ ♫
7046,Elliot Ziegler,NaN,Keep Me,21rxhltWEcMhJmEaSbma3z,56,False,2018,
7589,John Williams,NaN,Theme From Jurassic Park,2TZbQZXOuR8osP2AK8yYMN,63,False,1993,


In [ ]:
pd.set_option('display.max_rows', 10)